In [3]:
import numpy as np
import idx2numpy
import matplotlib.pyplot as plt

# Load Data

In [10]:
train_img = idx2numpy.convert_from_file('data/train-images.idx3-ubyte')
train_label = idx2numpy.convert_from_file('data/train-labels.idx1-ubyte')
test_img = idx2numpy.convert_from_file('data/t10k-images.idx3-ubyte')
test_label = idx2numpy.convert_from_file('data/t10k-labels.idx1-ubyte')

In [46]:
def load_data_of_digit(digit, imgs, lbs):
    idx = lbs==digit
    return imgs[idx]

In [47]:
img_1 = load_data_of_digit(1, train_img, train_label)
img_7 = load_data_of_digit(7, train_img, train_label)

# Model

In [51]:
class QuadraticModel:
    def __init__(self, in_features):
        self.in_features = in_features
        self.W = np.random.rand(in_features, in_features)
        self.v = np.random.rand(in_features)
        self.b = np.random.rand(1)
    
    def __call__(self, X):
        return np.reshape(np.sum(X * (X @ self.W), axis=-1) + X @ self.v + self.b, [-1, 1])
    
    def test_func(self, X, y):
        return y * self(X) # bx1 * bxd = bxd

    def _w(self, X):
        X1 = np.expand_dims(X, 1)
        X2 = np.expand_dims(X, -1)
        return X1 * X2, X, np.ones([len(X), 1])
    
    @property
    def w(self):
        return self.W, self.v, self.b

In [52]:
q = QuadraticModel(10)
x = np.random.rand(30, 10)
q(x)

array([[ 8.30101824],
       [18.51140928],
       [20.11054707],
       [13.35529962],
       [10.59260836],
       [14.26592825],
       [16.65403631],
       [12.19653204],
       [13.55155323],
       [13.35332794],
       [23.00342792],
       [21.54494284],
       [17.9913549 ],
       [10.23108107],
       [20.74742821],
       [ 9.08638757],
       [18.18391304],
       [14.75864826],
       [23.61544652],
       [17.88369707],
       [12.3493116 ],
       [19.79821076],
       [17.73685936],
       [17.40864584],
       [ 6.69096447],
       [11.70915029],
       [16.38579398],
       [12.30271294],
       [14.83909165],
       [ 9.39128553]])

In [57]:
q._w(x)[2].shape

(30, 1)

# Loss

In [ ]:
class Loss:
    def __init__(self, data, label, batch_size, lam=1e-3):
        self.data = data
        self.label = label
        self.n = batch_size
        self.lam = lam
        
    def __call__(self, model):
        X, y = self.sample_batch()
        q = model.test_func(X, y)
        W, v, b = model.w
        L = np.mean(np.log(1 + np.exp(-q))) + 0.5 * self.lam * (np.sum(W ** 2) + np.sum(v ** 2) + np.sum(b ** 2))
        L_w = self._w(X, y)
        return L, L_w
    
    def _w(self, model, X, y):
        q = model.test_func(X, y) # bx1
        coef = - y * np.exp(-q) / (1 + np.exp(-q)) # bx1
        W, v, b = model.w
        f_W, f_v, f_b = model._w(X) # bxdxd, bxd, bx1
        dW = np.mean(coef[:, np.newaxis] * f_W, axis=0) + self.lam * W
        dv = np.mean(coef * f_v, axis=0) + self.lam * v
        db = np.mean(coef * f_b, axis=0) + self.lam * b
        return dW, dv, db
    
    def sample_batch(self):
        if self.n:
            idx = np.random.choice(self.n)
            X = self.data[idx]
            y = self.label[idx]
            return X, y
        else:
            return self.data, self.label

# Optimizers

In [ ]:
class _Optimizer:
    def __init__(self, solution, alpha, silent=True):
        self.solution = solution
        self.alpha = alpha
        self.silent = silent

    @property
    def parameters(self):
        return self.solution.N.w


class SGD(_Optimizer):    
    def __call__(self, loss):
        f, g, _, _ = loss(self.solution)
        self.parameters[...] = self.parameters - self.alpha * g
        if not self.silent: return f, g
    
    def __str__(self) -> str: return 'SGD'


class Nesterov(_Optimizer):
    def __init__(self, solution, alpha, silent=True):
        super().__init__(solution, alpha, silent)
        self.k = 0
        self.x = np.copy(self.parameters)
        
    def __call__(self, loss):
        f, g, _, _ = loss(self.solution)
        x_ = self.parameters - self.alpha * g
        self.parameters[...] = (1 + self.mu) * x_ - self.mu * self.x
        self.x = x_
        if not self.silent: return f, g 

    @property
    def mu(self):
        return 1 - 3 / (5 + self.k)
    
    def __str__(self) -> str: return 'Nesterov'


class Adam(_Optimizer):
    def __init__(self, solution, alpha, beta_1=0.9, beta_2=0.999, epsilon=1e-8, silent=True):
        super().__init__(solution, alpha, silent)
        self.beta_1, self.beta_2 = beta_1, beta_2
        self.epsilon = epsilon
        self.m = 0
        self.v = 0
        self.k = 0
    
    def __call__(self, loss):
        self.k += 1
        f, g, _, _ = loss(self.solution)
        self.m = self.beta_1 * self.m + (1-self.beta_1) * g
        self.v = self.beta_2 * self.v + (1-self.beta_2) * (g ** 2)
        m_hat = self.m / (1 - self.beta_1 ** self.k)
        v_hat = self.v / (1 - self.beta_2 ** self.k)
        self.parameters[...] = self.parameters - self.alpha * m_hat / (np.sqrt(v_hat) + self.epsilon)
        if not self.silent: return f, g
    
    def __str__(self) -> str: return 'Adam'